In [1]:
## Installation required when using Google Colab
!pip install webcolors
!pip install word2number
!pip install -r requirements.txt

  Created wheel for word2number: filename=word2number-1.1-cp36-none-any.whl size=5588 sha256=611bd0884dfcab69b8aeb8ff11588917f134f6f3f90c688cfc436db52e88b817
  Stored in directory: /root/.cache/pip/wheels/46/2f/53/5f5c1d275492f2fce1cdab9a9bb12d49286dead829a4078e0e
Successfully built word2number
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [0]:
## Imports
import spacy
from spacy import displacy
import webcolors
from word2number import w2n

In [0]:
## Object to be used in Sketching module. Contains the necessary details 
# Find another name?
class Object:
  def __init__(self, name, color="white", size=1, number = 1, location="random"):
    self.name = name
    self.color = color
    self.size = size
    self.number = number
    self.location = location
  
  def print(self):
    print("Name: ",self.name,"\tColor:",self.color," Size:",self.size," Number:",self.number," Location:",self.location)

In [0]:
## Functions to identify what information a feature gives about the object
def isColor(feature):
  return feature in webcolors.CSS3_NAMES_TO_HEX

def isNumber(feature):
  try:
    w2n.word_to_num(feature)
    return True
  except:
    return False

In [0]:
## Extract the relevant information of the main object
# Implement better ->
def extract_features(doc):
  ## Object of interest, dependency tag = "attr"
  obj = [token for token in doc if (token.dep_ == "attr")][0]

  stack = []
  feature_lst = []
  stack=([child for child in obj.children])

  while stack:
    token = stack.pop()
    feature_lst.append(token.lemma_)
    if(token.pos_ != "NOUN"):
      stack+=[child for child in token.children]
  
  return feature_lst

## Creates an "Object" based on the given information
def match_features(feature_lst):
  ob = Object(name = obj.text)
  for feature in feature_lst:
    if(isColor(feature)):
      ob.color = feature
    elif(isNumber(feature)):
      number = w2n.word_to_num(feature)
      ob.number = number
  return ob

In [56]:
## MAIN ##

nlp = spacy.load('en_core_web_sm')

# Get a sentence input from the user - to be deleted when merged with Speech module
sentence = input("Enter a sentence: ")

# Store the sentence as a Spacy Doc object. Contains information on the tokens
doc = nlp(sentence)

obj = match_features(extract_features(doc))
obj.print()

Enter a sentence: There's a blue bird on the roof of the house.
Name:  bird 	Color: blue  Size: 1  Number: 1  Location: random


In [0]:
## Some functions to help analyzing the sentence

# Lemma: Base form of the token, with no inflectional suffixes
# Part of Speech: Coarse-grained part-of-speech
# Tag: Fine-grained part-of-speech
# Dep: Syntactic dependency relation

def display_tokens(doc):
  start = '\033[36m'
  end = "\033[0;0m"
  print(start.ljust(45),"Lemma PoS Tag Dep",end)

  for token in doc:
    print(token.text.ljust(40) ,token.lemma_, token.pos_, token.tag_, token.dep_.ljust(30),"\t",spacy.explain(token.dep_))

def analyze_dependency(doc):
  for token in doc: 
    print("Token:",token,"\n\tAncestors: ",[ancestor for ancestor in token.ancestors],"\n\tChildren: ",[child for child in token.children])
    print("-------------------")

def dependency_tree(doc):
  return displacy.render(doc, jupyter=True, style='dep')

In [29]:
display_tokens(doc)

                                         Lemma PoS Tag Dep 
There                                    there ADV EX expl                           	 expletive
's                                       be VERB VBZ ROOT                           	 None
a                                        a DET DT det                            	 determiner
bird                                     bird NOUN NN attr                           	 attribute
on                                       on ADP IN prep                           	 prepositional modifier
the                                      the DET DT det                            	 determiner
roof                                     roof NOUN NN pobj                           	 object of preposition
of                                       of ADP IN prep                           	 prepositional modifier
the                                      the DET DT det                            	 determiner
house                                    house